In [1]:
from train import *

saver = tf.train.Saver()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


In [2]:
tf.global_variables()

[<tf.Variable 'w_embed:0' shape=(10000, 100) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/fw/gru_cell/gates/kernel:0' shape=(250, 300) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/fw/gru_cell/gates/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/fw/gru_cell/candidate/kernel:0' shape=(250, 150) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/fw/gru_cell/candidate/bias:0' shape=(150,) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/bw/gru_cell/gates/kernel:0' shape=(250, 300) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/bw/gru_cell/gates/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/bw/gru_cell/candidate/kernel:0' shape=(250, 150) dtype=float32_ref>,
 <tf.Variable 'bidirectional_rnn/bw/gru_cell/candidate/bias:0' shape=(150,) dtype=float32_ref>,
 <tf.Variable 'w_omega:0' shape=(300, 50) dtype=float32_ref>,
 <tf.Variable 'b_omega:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'u_omega:0' shape=(50,) d

In [24]:

# Calculate alpha coefficients for the first test example
with tf.Session() as sess:
    saver.restore(sess, MODEL_PATH)

    x_batch_test, y_batch_test = X_test[0:10], y_test[0:10]
    seq_len_test = np.sum(x_batch_test !=0, axis=1) # actual lengths of sequences
    alphas_test = sess.run([alphas], feed_dict={batch_ph: x_batch_test, target_ph: y_batch_test,
                                                keep_prob_ph: 1.0,
                                                batch_size_ph:len(x_batch_test)})

INFO:tensorflow:Restoring parameters from ./model_dir/model


In [50]:
y_batch_test

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

In [29]:
alphas_test[0].shape

(10, 250)

In [38]:
# Build correct mapping from word to index and inverse
word_index = imdb.get_word_index()
word_index = {word: index + INDEX_FROM for word, index in word_index.items()}
word_index[":PAD:"] = 0
word_index[":START:"] = 1
word_index[":UNK:"] = 2
index_word = {value: key for key, value in word_index.items()}
# Represent the sample by words rather than indices
WORDS = list([list(map(index_word.get, x)) for x in x_batch_test ])

In [79]:
print(WORDS[0])

[':START:', 'please', 'give', 'this', 'one', 'a', 'miss', 'br', 'br', ':UNK:', ':UNK:', 'and', 'the', 'rest', 'of', 'the', 'cast', 'rendered', 'terrible', 'performances', 'the', 'show', 'is', 'flat', 'flat', 'flat', 'br', 'br', 'i', "don't", 'know', 'how', 'michael', 'madison', 'could', 'have', 'allowed', 'this', 'one', 'on', 'his', 'plate', 'he', 'almost', 'seemed', 'to', 'know', 'this', "wasn't", 'going', 'to', 'work', 'out', 'and', 'his', 'performance', 'was', 'quite', ':UNK:', 'so', 'all', 'you', 'madison', 'fans', 'give', 'this', 'a', 'miss', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD:', ':PAD

In [67]:
alphas_test

[array([[6.23109145e-03, 7.92000908e-03, 8.31867289e-03, ...,
         1.58739625e-04, 1.73065317e-04, 1.92631618e-04],
        [3.91616207e-03, 3.96514591e-03, 3.32271331e-03, ...,
         1.19216675e-02, 1.21548232e-02, 1.25672687e-02],
        [2.46916991e-03, 2.41376786e-03, 2.59333150e-03, ...,
         9.21661872e-03, 1.04165031e-02, 1.10096373e-02],
        ...,
        [2.78337742e-04, 2.87979608e-04, 2.94701458e-04, ...,
         4.67624523e-05, 5.09756428e-05, 5.67466595e-05],
        [4.58475715e-03, 4.27286187e-03, 4.44687391e-03, ...,
         1.12041284e-03, 1.22175773e-03, 1.36021606e-03],
        [3.79521237e-03, 4.00373898e-03, 3.38824792e-03, ...,
         5.68107329e-03, 4.63395612e-03, 5.79124503e-03]], dtype=float32)]

In [91]:
# Save visualization as HTML
with open("visualization.html", "a") as html_file:
    html_file.write('<head>\n<meta charset="UTF-8">\n</head>\n')
    for words, alphas, y in zip(WORDS, alphas_test[0], y_batch_test):
        html_file.write('<font style="background: rgb(255, 255, 250)">%s</font> ' % (y))    
        for word, alpha in zip(words, alphas / alphas.max()):
            if word == ":START:":
                continue
            elif word == ":PAD:":
                continue
            if y ==1:
                html_file.write('<font style="background: rgba(95, 95, 250, %f)">%s</font>\n' % (alpha, word))
            else:
                html_file.write('<font style="background: rgba(250, 95, 95, %f)">%s</font>\n' % (alpha, word))
        html_file.write('<br>')
print('\nOpen visualization.html to checkout the attention coefficients visualization.')


Open visualization.html to checkout the attention coefficients visualization.
